# Visualización de incendios con Folium en Jupyter Notebook

Este notebook desarrolla una aplicación visual con Folium para visualizar incendios usando un dataset de incendios.

In [ ]:
# Instalación de librerías necesarias
!pip install folium pandas

In [ ]:
# Importación de librerías
import folium
import pandas as pd
from folium.plugins import HeatMap, MarkerCluster

# Carga del dataset
df = pd.read_csv('fires_map.csv')

# Inspección inicial
df.head()

## Mapa 1: Mapa de calor
Se crea un mapa de calor para visualizar las zonas con mayor cantidad de incendios.

In [ ]:
# Creación del mapa de calor
heat_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)
heat_data = df[['latitude', 'longitude']].values.tolist()
HeatMap(heat_data).add_to(heat_map)
heat_map

## Mapa 2: Marcadores de incendios con popups
Cada marcador tendrá un color dependiendo del brillo (`brightness`).

In [ ]:
# Creación del mapa con marcadores
marker_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)

for _, row in df.iterrows():
    if row['brightness'] < 330:
        color = 'yellow'
    elif row['brightness'] < 360:
        color = 'orange'
    else:
        color = 'red'

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Brillo: {row['brightness']}",
        icon=folium.Icon(color=color, icon='fire')
    ).add_to(marker_map)

marker_map

## Mapa 3: Agrupación con MarkerCluster
Se agrupan los incendios cercanos en clusters.

In [ ]:
# Creación del mapa con MarkerCluster
cluster_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)
marker_cluster = MarkerCluster().add_to(cluster_map)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Brillo: {row['brightness']}"
    ).add_to(marker_cluster)

cluster_map

## Mapa 4: Diferenciación de incendios por satélite
Se asignan colores diferentes a los incendios según el satélite que los detectó.

In [ ]:
# Creación del mapa con control de capas por satélite
satellite_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)
aqua_group = folium.FeatureGroup(name='Aqua').add_to(satellite_map)
terra_group = folium.FeatureGroup(name='Terra').add_to(satellite_map)

for _, row in df.iterrows():
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Brillo: {row['brightness']}"
    )
    if row['satellite'] == 'Aqua':
        marker.add_to(aqua_group)
    else:
        marker.add_to(terra_group)

folium.LayerControl().add_to(satellite_map)
satellite_map

## Mapa 5: Control de capas por intensidad de brillo
Cada capa representa incendios con diferentes niveles de brillo.

In [ ]:
# Creación del mapa con capas por brillo
brightness_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)
low_brightness = folium.FeatureGroup(name='Brillo Bajo').add_to(brightness_map)
medium_brightness = folium.FeatureGroup(name='Brillo Medio').add_to(brightness_map)
high_brightness = folium.FeatureGroup(name='Brillo Alto').add_to(brightness_map)

for _, row in df.iterrows():
    if row['brightness'] < 330:
        row_group = low_brightness
    elif row['brightness'] < 360:
        row_group = medium_brightness
    else:
        row_group = high_brightness
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Brillo: {row['brightness']}"
    ).add_to(row_group)

folium.LayerControl().add_to(brightness_map)
brightness_map